In [2]:
import pandas as pd
import numpy as np

In [6]:
genome = pd.read_csv('genome.csv', on_bad_lines='skip')
genome.rename(columns={'symbol':'gene'}, inplace=True)
slim_genome = genome[['gene','#"chrom"', 'chromStart', 'chromEnd', 'location']]
slim_genome

<ipython-input-6-6f1cbe8ff997>:1: DtypeWarning: Columns (26,39,44,50,56) have mixed types. Specify dtype option on import or set low_memory=False.
  genome = pd.read_csv('genome.csv', on_bad_lines='skip')


,gene,"#""chrom""",chromStart,chromEnd,location
0,DDX11L1,chr1,11873,14408,1p36.33
1,WASH7P,chr1,14361,29369,1p36.33
2,MIR6859-1,chr1,17368,17435,1p36.33
3,MIR1302-2HG,chr1,29773,35417,1p36.33
4,MIR1302-2,chr1,30365,30502,1p36.33
...,...,...,...,...,...
43439,RNA45SN4,chr22_KI270733v1_random,122272,135644,22 unplaced
43440,RNA18SN4,chr22_KI270733v1_random,125930,127798,22 unplaced
43441,RNA5-8SN4,chr22_KI270733v1_random,128876,129031,22 unplaced
43442,RNA28SN4,chr22_KI270733v1_random,130203,135279,22 unplaced


In [7]:
#Here you can import your dataframe where each row is a seperate gene with corresponding copy number
ccle_data = pd.read_csv('/content/CCLE_fully_mapped_v4.csv', index_col=0)
ccle_data

,cell_line,AmpliconID,Classification,gene,cn,ge,State,State_annotation (salient pathways for the state),Salient_genes_in_the_state,karyotyping,AA prediction,CH prediction
0,HCC2157,2.0,ecDNA,SLFNL1,2.031471,1.778209,CCLE_F0,partial-EMT; Notch signaling,"ELF3, GRHL2, CLDN4, DSP, CGN, DSC2, ALDH1A3, J...",N,Y,negative
1,HCC2157,2.0,ecDNA,CTPS1,2.031471,7.022257,CCLE_F0,partial-EMT; Notch signaling,"ELF3, GRHL2, CLDN4, DSP, CGN, DSC2, ALDH1A3, J...",N,Y,negative
2,HCC2157,2.0,ecDNA,CITED4,2.031471,6.574404,CCLE_F0,partial-EMT; Notch signaling,"ELF3, GRHL2, CLDN4, DSP, CGN, DSC2, ALDH1A3, J...",N,Y,negative
3,HCC2157,2.0,ecDNA,SLFNL1-AS1,2.031471,NaN,CCLE_F0,partial-EMT; Notch signaling,"ELF3, GRHL2, CLDN4, DSP, CGN, DSC2, ALDH1A3, J...",N,Y,negative
4,HCC2157,2.0,ecDNA,SCMH1,1.571024,6.584963,CCLE_F0,partial-EMT; Notch signaling,"ELF3, GRHL2, CLDN4, DSP, CGN, DSC2, ALDH1A3, J...",N,Y,negative
...,...,...,...,...,...,...,...,...,...,...,...,...
678,BT20,13.0,ecDNA,LINC00544,NaN,NaN,CCLE_F6,MAGE protein family,"MAGEA2, MAGEA2B, MAGEA3,MAGEA6, MAGEA12",NaN,Y,NaN
679,BT20,13.0,ecDNA,PRDX2P1,NaN,NaN,CCLE_F6,MAGE protein family,"MAGEA2, MAGEA2B, MAGEA3,MAGEA6, MAGEA12",NaN,Y,NaN
680,BT20,13.0,ecDNA,WDR95P,NaN,NaN,CCLE_F6,MAGE protein family,"MAGEA2, MAGEA2B, MAGEA3,MAGEA6, MAGEA12",NaN,Y,NaN
681,BT20,13.0,ecDNA,LINC01058,NaN,NaN,CCLE_F6,MAGE protein family,"MAGEA2, MAGEA2B, MAGEA3,MAGEA6, MAGEA12",NaN,Y,NaN


In [8]:
ccle_mapped = pd.merge(ccle_data, slim_genome, on='gene', how='left')

In [10]:
#The logic of script is, for a given cell line, sort by copy number. For that copy number, determine all genes found and start and end intervals

#Below is the simplified version of the script, it simply note that it does not account for the possability of two genes having identical copy numbers and being found on different chromosomes
def interval_finder(dataframe, cell_line: str):
  cn_groupby = dataframe.groupby('cn')
  new_dict = {'interval_size_mbp': [],
              'locations': [],
              'genes': [],
              'copy_number': [],
              'ampID' : [],
              'interval_start': [],
              'interval_end': []
              }
  for cn, df in cn_groupby:
    first_instance = df.iloc[0]
    interval_start = df['chromStart'].min()
    interval_end = df['chromEnd'].max()
    locations = list(set(df['location'].to_list()))
    ampID = first_instance['AmpliconID']
    genes = list(set(df['gene']))
    new_dict['interval_size_mbp'].append((interval_end-interval_start)/1000000)
    new_dict['locations'].append(locations)
    new_dict['genes'].append(genes)
    new_dict['copy_number'].append(cn)
    new_dict['ampID'].append(ampID)
    new_dict['interval_start'].append(interval_start)
    new_dict['interval_end'].append(interval_end)
  new_df = pd.DataFrame(new_dict)
  new_df['cell_line'] = cell_line
  return new_df.sort_values('copy_number', ascending=False)


interval_finder(ccle_mapped[ccle_mapped['cell_line'] == 'HCC1599'], 'HCC1599')

,interval_size_mbp,locations,genes,copy_number,ampID,interval_start,interval_end,cell_line
11,1.210269,[19q12],"[C19orf12, CCNE1, ZNF536, URI1, LINC01834]",2.741230,1.0,29698885.0,30909154.0,HCC1599
10,0.097110,[19q13.11],"[RGS9BP, SNORA68B, PDCD5, ANKRD27]",2.573435,1.0,32581189.0,32678299.0,HCC1599
9,0.137759,[19q13.11],"[NUDT19, TDRD12]",2.538217,1.0,32691820.0,32829579.0,HCC1599
8,0.039256,[19q13.11],[SLC7A9],2.503013,1.0,32830510.0,32869766.0,HCC1599
7,0.956040,"[19q13.11, 19q12]","[LINC01533, TSHZ3, LINC01782, LINC01791, LINC0...",2.453080,1.0,31149875.0,32105915.0,HCC1599
6,0.102304,[19q13.11],"[CEP89, FAAP24]",2.437630,1.0,32875924.0,32978228.0,HCC1599
5,0.140280,[19q13.11],"[DPY19L3, ZNF507]",2.403426,1.0,32345609.0,32485889.0,HCC1599
4,0.086296,[19q13.11],[RHPN2],2.376814,1.0,32978591.0,33064887.0,HCC1599
3,0.010016,[19q12],[PLEKHF1],2.183177,1.0,29665460.0,29675476.0,HCC1599
2,0.069719,[3q25.31],[LINC00886],1.914930,4.0,156747342.0,156817061.0,HCC1599
